# Treinando Sentence-BERT com Cosine Similarity Loss

Neste notebook, vamos treinar um modelo **Sentence-BERT (sBERT)** para medir similaridade entre sentenças usando a **Cosine Similarity Loss**.

O objetivo é fazer o modelo aprender a **aproximar embeddings** de sentenças semelhantes e **afastar embeddings** de sentenças diferentes.

## 1. Instalação e importação das bibliotecas

In [1]:
import os
import torch
import pandas as pd
import accelerate, transformers
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader, Dataset

Se você tiver acesso a uma **GPU** local, irá aparecer o nome abaixo:

In [2]:
print("CUDA disponível:", torch.cuda.is_available())
print("Dispositivo atual:", torch.cuda.current_device() if torch.cuda.is_available() else "CPU")
print("Nome da GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Nenhuma GPU detectada")

CUDA disponível: True
Dispositivo atual: 0
Nome da GPU: NVIDIA GeForce GTX 1650


## 2. Carregar o dataset de treino

O arquivo `data/train.csv` contém pares de sentenças e um valor de similaridade (`similarity_score`) entre 0 e 5.

In [3]:
df = pd.read_csv("../data/train.csv")
df.head()

,sentence1,sentence2,similarity_score
0,A woman puts make-up on.,A woman is putting on eyeshadow.,3.333
1,Scientists prove there is water on Mars,Has Nasa discovered water on Mars?,2.000
2,Ban Ki-moon to Review Syria Chemical Arms Accord,Ban to review Syria chemical arms accord,3.600
3,the 5 permanent united nations security counci...,"p5+1 members are britain, china, france, russi...",3.600
4,Japan PM calls for quicker tsunami waste proce...,Japan marks one year since quake and tsunami d...,1.000


## 3. Pré-processamento

Normaliza o `similarity_score`, pois os valores vão de 0-5, garantindo os valores na escala original.

In [4]:
df['similarity_score'] = df['similarity_score'] / 5.0

## 4. Preparar os exemplos para treino

O Sentence-BERT espera exemplos no formato `InputExample`, contendo dois textos e uma label (score de similaridade).

In [5]:
train_examples = [
    InputExample(
        texts=[row['sentence1'], row['sentence2']],
        label=float(row['similarity_score'])
    )
    for _, row in df.iterrows()
]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)  # type: ignore
print(f"✅ Total de exemplos de treino: {len(train_examples)}")

✅ Total de exemplos de treino: 4832


## 5. Carregar o modelo sBERT

Usaremos o modelo `all-MiniLM-L12-v2`, que é bem eficiente para tarefas de similaridade semântica.
Obs.: Também pode testar com o modelo `all-MiniLM-L6-v2`

Também ativa o treinamento pela GPU, se estiver disponível.

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('all-MiniLM-L12-v2', device=device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 6. Definir a função de perda CosineSimilarityLoss

A função `CosineSimilarityLoss` faz o modelo maximizar a similaridade do cosseno entre embeddings de sentenças semelhantes e minimizar a de sentenças diferentes.

In [7]:
train_loss = losses.CosineSimilarityLoss(model=model)

## 7. Treinar o modelo

In [8]:
output_dir = '../output/sbert-cosine'
os.makedirs(output_dir, exist_ok=True)

optimizer_params: dict[str, object] = {'lr': 3e-5}
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=10,                          # número de épocas no treinamento
    warmup_steps=100,                   # número de passos para aquecimento, evita overfitting
    output_path=output_dir,             # diretório para salvar o modelo
    show_progress_bar=True,             # mostra barra de progresso
    optimizer_params=optimizer_params   # otimização com taxa de aprendizado personalizada
)

print(f"✅ Treinamento concluído! Modelo salvo em: {output_dir}")

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.015100
1000,0.005200
1500,0.003300


✅ Treinamento concluído! Modelo salvo em: ../output/sbert-cosine


## Conclusão

O modelo foi treinado com **Cosine Similarity Loss**, que aprende a prever o grau contínuo de similaridade entre pares de sentenças.

No próximo notebook, vamos **avaliar** esse modelo no conjunto de teste (`test.csv`) e comparar com o `ground_truth.csv`."